# Importing Libraries

In [1]:
import pandas as pd
from pathlib import Path

## File path and dataframes

In [2]:
# Directories where all subdirectories containing excel files are stored
current_dir = Path().resolve()

In [3]:
# Data frame to store results
results_MT_df = pd.DataFrame(columns=['Directory', 'MT_Ready', 'Percent', 'MT_Not_Ready', 'Percent'])
results_LT_df = pd.DataFrame(columns=['Directory', 'LT_Ready', 'Percent', 'LT_Not_Ready', 'Percent'])

### Functions to find files, driectories, calculate MT or LT readiness

In [4]:
# Function to find HT, Mt and LT files in the sub-directories
def find_file(directory, keyword):
    for file in directory.glob(f'*{keyword}*.xlsx'):
        return file
    return None

# Function to process each directory containing oversizing results to determine its effect on MT and LT readiness. 
def process_directory(directory):
    # Find path to the excel files in the current directory
    HT_data_path = find_file(directory, 'HT')
    MT_data_path = find_file(directory, 'MT')
    LT_data_path = find_file(directory, 'LT')

    if not HT_data_path or not MT_data_path or not LT_data_path:
        print(f"Missing HT or MT or LT files in the directory: {directory}")
    return HT_data_path, MT_data_path, LT_data_path

# Function to count for MT readiness
def MT_readiness(directory, HT_data_path, MT_data_path):
    # Load the data from excel
    HT_df = pd.read_excel(HT_data_path)
    MT_df = pd.read_excel(MT_data_path)
    
    # sample number as column index 
    HT_df.set_index('Sample Number', inplace = True)
    MT_df.set_index('Sample Number', inplace = True)

    # Define your threshold
    threshold_cold_hours = 0  # adjust as needed
    threshold_shd = 0

    # Initialise counters
    ready_count = 0
    not_ready_count = 0

    # total number of samples
    size = len(HT_df)

    # Loop through each sample
    for sample in HT_df.index:
        # Get heating demand and cold hours for each sample in HT supply
        HT_heating_demand = HT_df.loc[sample, 'Heating_demand']
        HT_cold_hours = HT_df.loc[sample, 'Underheated_hours']
    
        # Get heating demand and cold hours for each sample in MT supply
        MT_heating_demand = MT_df.loc[sample, 'Heating_demand']
        MT_cold_hours = MT_df.loc[sample, 'Underheated_hours']
    
        # Check for LTH readiness
        # if heating demands or cold hours in MT supply are higher than those in HT supply
        # then sample is "not ready: 0" for MT
    
        if abs(MT_heating_demand - HT_heating_demand) < threshold_shd or abs(MT_cold_hours - HT_cold_hours) > threshold_cold_hours:
            not_ready_count = not_ready_count + 1 # number of Not ready
            # if not then the sample is "ready = 1" for MT
        else:
            ready_count = ready_count + 1

        not_ready_percent = round((not_ready_count/size)*100)
        ready_percent = round(100 - not_ready_percent)

    # Add results to the results DataFrame
    results_MT_df.loc[len(results_MT_df)] = [directory.name, ready_count, ready_percent, not_ready_count, not_ready_percent]

    return results_MT_df

# Function to count for LT readiness
def LT_readiness(directory, HT_data_path, LT_data_path):
    # Load the data from excel
    HT_df = pd.read_excel(HT_data_path)
    LT_df = pd.read_excel(LT_data_path)
    
    # sample number as column index 
    HT_df.set_index('Sample Number', inplace = True)
    LT_df.set_index('Sample Number', inplace = True)

    # Define your threshold
    threshold_cold_hours = 0  # adjust as needed
    threshold_shd = 0

    # Initialise counters
    ready_count = 0
    not_ready_count = 0

    # total number of samples
    size = len(HT_df)

    # Loop through each sample
    for sample in HT_df.index:
        # Get heating demand and cold hours for each sample in HT supply
        HT_heating_demand = HT_df.loc[sample, 'Heating_demand']
        HT_cold_hours = HT_df.loc[sample, 'Underheated_hours']
    
        # Get heating demand and cold hours for each sample in LT supply
        LT_heating_demand = LT_df.loc[sample, 'Heating_demand']
        LT_cold_hours = LT_df.loc[sample, 'Underheated_hours']
    
        # Check for LTH readiness
        # if heating demands or cold hours in LT supply are higher than those in HT supply
        # then sample is "not ready: 0" for LT
    
        if abs(LT_heating_demand - HT_heating_demand) < threshold_shd or abs(LT_cold_hours - HT_cold_hours) > threshold_cold_hours:
            not_ready_count = not_ready_count + 1 # number of Not ready
            # if not then the sample is "ready = 1" for MT
        else:
            ready_count = ready_count + 1

        not_ready_percent = round((not_ready_count/size)*100)
        ready_percent = round(100 - not_ready_percent)

    # Add results to the results DataFrame
    results_LT_df.loc[len(results_LT_df)] = [directory.name, ready_count, ready_percent, not_ready_count, not_ready_percent]

    return results_LT_df

### Read all the directories in the current directory containing oversizing studies

In [5]:
current_dir = Path().resolve()
# Loop through each subdirectory in the current directory
for subdir in current_dir.iterdir():
    # Only study a directory and avoid other files
    if subdir.is_dir() and not subdir.name.startswith('.'):
        # Recive the excel file paths
        HT_path, MT_path, LT_path = process_directory(subdir)
        # Process data only if the paths are not empty
        if HT_path and MT_path:
            MT_ready = MT_readiness(subdir, HT_path, MT_path)
        if HT_path and LT_path:
            LT_ready = LT_readiness(subdir, HT_path, LT_path)

In [6]:
# The effect of ovesizing on MT readiness for the represented samples 
MT_ready

,Directory,MT_Ready,Percent,MT_Not_Ready,Percent
0,Data_generic_ovs,926,71,374,29
1,Data_ovs_1pt25,1235,95,65,5
2,Data_ovs_1pt66,1292,99,8,1
3,Data_ovs_2pt5,1293,99,7,1
4,Data_ovs_5,1291,99,9,1


In [7]:
# The effect of ovesizing on MT readiness for the represented samples 
LT_ready

,Directory,LT_Ready,Percent,LT_Not_Ready,Percent
0,Data_generic_ovs,18,1,1282,99
1,Data_ovs_1pt25,123,9,1177,91
2,Data_ovs_1pt66,550,42,750,58
3,Data_ovs_2pt5,1258,97,42,3
4,Data_ovs_5,1289,99,11,1
